In [0]:
!wget https://uacrmq.bn.files.1drv.com/y4mT1jud_V_DEwauFGLYrGeYH5Lxbig3sjNV_4P3eKAvrd-qtgfWCUa75glkj1CfjoTDzd2J-ZvATQFKkNgKg5Vc_b0QFL1_LmVj_Gts22ak3nQMeQ_zACmLdjbFAruKcQurwgpABqWP8NcGV6E_ayYzOTD6yIIsJe75Vd5yirDKAL6h4Yqf5smXNsrbk37vf1O/input.csv

--2018-07-30 08:30:09--  https://uacrmq.bn.files.1drv.com/y4mT1jud_V_DEwauFGLYrGeYH5Lxbig3sjNV_4P3eKAvrd-qtgfWCUa75glkj1CfjoTDzd2J-ZvATQFKkNgKg5Vc_b0QFL1_LmVj_Gts22ak3nQMeQ_zACmLdjbFAruKcQurwgpABqWP8NcGV6E_ayYzOTD6yIIsJe75Vd5yirDKAL6h4Yqf5smXNsrbk37vf1O/input.csv
Resolving uacrmq.bn.files.1drv.com (uacrmq.bn.files.1drv.com)... 204.79.197.213
Connecting to uacrmq.bn.files.1drv.com (uacrmq.bn.files.1drv.com)|204.79.197.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399393515 (381M) [application/octet-stream]
Saving to: ‘input.csv.1’

input.csv.1         100%[===================>] 380.89M  25.1MB/s    in 15s     

2018-07-30 08:30:25 (25.6 MB/s) - ‘input.csv.1’ saved [399393515/399393515]



In [0]:
!ls

datalab  input.csv


In [0]:
!pip install gensim
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
import nltk
from gensim.corpora import Dictionary
# preprocessing imports
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# model imports
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# hyperparameter training imports
from sklearn.model_selection import GridSearchCV

# visualization imports

import seaborn as sns
import base64
import io
%matplotlib inline

Using TensorFlow backend.


In [0]:
data = pd.read_csv("input.csv")

In [0]:
for i in range(data.shape[0]):
    rmtext = data.author[i].split(' ')
    for text in rmtext:
        data.context[i] = data.context[i].replace(text,'')

data['context'] = data['context'].str.replace('\r', '')
data['context'] = data['context'].str.replace('\n', ' ')
data['context'] = data['context'].str.replace("\'s", '')
data['context'] = data['context'].str.replace("\'s", '')
data['context'] = data['context'].str.replace("\'", '')
data['context'] = data['context'].str.replace("'", '')
# def remove_non_ascii(text):
#     return ''.join(i for i in text if ord(i)<128)
 
#data['context'] = data['context'].apply(remove_non_ascii)

from nltk.corpus import stopwords
# stop = stopwords.words('english')
# test['tweet'].apply(lambda x: [item for item in x if item not in stop])
data.head()

,author,context
0,Andrew Lang,A Collection of Ballads Contents: Sir Pa...
1,Andrew Lang,A MONK OF FIFE Being the Chronicle written ...
2,Andrew Lang,A SHORT HISTORY OF SCOTLAND CHAPTER I. SCO...
3,Andrew Lang,ADVENTURES AMONG BOOKS by Contents: Pre...
4,Andrew Lang,ALFRED TENNYSON by INTRODUCTION In ...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data.context)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(data.context)
tf_feature_names = tf_vectorizer.get_feature_names()

In [0]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 15

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
house dont quite father room sir got captain work course
Topic 1:
ses ad wot im ginger arter sam ave er bob
Topic 2:
dick tom sam rover baxter dora putnam boys songbird crabtree
Topic 3:
roberts campbell willis mrs bemis lawton curwen _mrs mcilheny agnes
Topic 4:
literary literature boston lowell author new book longfellow read cambridge
Topic 5:
dave phil roger merwell nat haskers barringford jasniff jessie poole
Topic 6:
king prince princess queen palace prigio answered sultan fairy daughter
Topic 7:
king richard cuthbert earl edward duke henry english france england
Topic 8:
mr mrs miss dont lady sir im room gentleman wife
Topic 9:
holmes watson sherlock lestrade mr gregson mortimer room mycroft inspector
Topic 10:
illustration children garden bed hayloft child marching land sea play
Topic 11:
alan ye catriona balfour prestongrange says stewart james andie david
Topic 12:
shep whopper snap giant spink jed ham lake felps sanborn
Topic 13:
skipper mate ll ve don cook didn dec

In [0]:
from gensim.summarization.summarizer import summarize

In [0]:
# data['sumarize'] = np.nan
data['sumarize'] = data['context'].apply(summarize)


NameError: ignored

In [0]:
data['sumarize'].head()

NameError: ignored